[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/nn_adv/class_02/Laborat%C3%B3rios/lab_utils_ml_adv_week_2.zip
      
!unzip -n -q lab_utils_ml_adv_week_2.zip

In [ ]:
# Testar se estamos no Google Colab
# Necessário para ativar widgets
try:
  import google.colab
  IN_COLAB = True
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  IN_COLAB = False

# Função Softmax
Neste laboratório, exploraremos a função softmax. Essa função é usada na regressão softmax e em redes neurais para resolver problemas de classificação multiclasse. 

<center>  <img  src="./images/C2_W2_Softmax_Header.PNG" width="600" />  <center/>

  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from IPython.display import display, Markdown, Latex
from sklearn.datasets import make_blobs
%matplotlib widget
from matplotlib.widgets import Slider
from lab_utils_common import dlc
from lab_utils_softmax import plt_softmax
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

**Nota**: Normalmente, neste curso, os notebooks usam a convenção de iniciar as contagens com 0 e terminar com N-1, $\sum_{i=0}^{N-1}$, enquanto as aulas teóricas começam com 1 e terminam com N, $\sum_{i=1}^{N}$. Isso se deve ao fato de que o código normalmente inicia a iteração com 0, enquanto na aula, contar de 1 a N resulta em equações mais limpas e sucintas. Este notebook tem mais equações do que o normal em um laboratório e, portanto, romperá com a convenção e contará de 1 a N.

## Introdução a Função Softmax
Tanto na regressão softmax quanto nas redes neurais com saídas Softmax, são geradas N saídas e uma saída é selecionada como a categoria prevista. Em ambos os casos, um vetor $\mathbf{z}$ é gerado por uma função linear que é aplicada a uma função softmax. A função softmax converte $\mathbf{z}$ em uma distribuição de probabilidade, conforme descrito abaixo. Depois de aplicar a softmax, cada saída estará entre 0 e 1 e as soma das saídas será igual a1, de modo que possam ser interpretadas como probabilidades. As entradas maiores corresponderão a saída com maiores probabilidades.



<center>  <img  src="./images/C2_W2_SoftmaxReg_NN.png" width="600" />  

A função softmax pode ser escrita como:
$$a_j = \frac{e^{z_j}}{ \sum_{k=1}^{N}{e^{z_k} }} \tag{1}$$

O resultado $\mathbf{a}$ é um vetor de comprimento N, portanto, para a regressão softmax, você também poderia escrever:

\begin{align}
\mathbf{a}(x) =
\begin{bmatrix}
P(y = 1 | \mathbf{x}; \mathbf{w},b) \\
\vdots \\
P(y = N | \mathbf{x}; \mathbf{w},b)
\end{bmatrix}
=
\frac{1}{ \sum_{k=1}^{N}{e^{z_k} }}
\begin{bmatrix}
e^{z_1} \\
\vdots \\
e^{z_{N}} \\
\end{bmatrix} \tag{2}
\end{align}


O que mostra que a saída é um vetor de probabilidades. A primeira entrada é a probabilidade de a entrada ser a primeira categoria dada a entrada $\mathbf{x}$ e os parâmetros $\mathbf{w}$ e $\mathbf{b}$.  
Vamos criar uma implementação do NumPy:

In [ ]:
def my_softmax(z):
    ez = np.exp(z)   #exponencial elemento-a-elemento
    sm = ez/np.sum(ez)
    return(sm)

Abaixo, varie os valores das entradas `z` usando os controles deslizantes.

In [ ]:
plt.close("all")
plt_softmax(my_softmax)

À medida que você está variando os valores dos z's acima, há alguns aspectos a serem observados:
* o exponencial no numerador do softmax amplia pequenas diferenças nos valores 
* Os valores de saída somam um
* o softmax abrange todos as saídas outputs. Uma alteração em `z0`, por exemplo, alterará os valores de `a0`-`a3`. Compare isso com outras ativações, como ReLU ou Sigmoid, que têm uma única entrada e uma única saída.

## Custo
<center> <img  src="./images/C2_W2_SoftMaxCost.png" width="400" />    <center/>

A função de perda associada ao Softmax, a perda de entropia cruzada, é:
\begin{equation}
  L(\mathbf{a},y)=\begin{cases}
    -log(a_1), & \text{if $y=1$}.\\
        &\vdots\\
     -log(a_N), & \text{if $y=N$}
  \end{cases} \tag{3}
\end{equation}

Onde y é a categoria de alve para este exemplo e $\mathbf{a}$ é a saída de uma função softmax. Em particular, os valores em $\mathbf{a}$ são probabilidades que somam um.
>**Lembre-se: neste curso, a perda é para um exemplo, enquanto o custo abrange todos os exemplos. 
 
 
Observe que em (3) acima, somente a linha que corresponde ao alvo contribui para a perda, as outras linhas são zero. Para escrever a equação de custo, precisamos de uma "função indicadora" que será 1 quando o índice corresponder ao alvo e zero caso contrário. 

  $$\mathbf{1}\{y == n\} = =\begin{cases}
  1, & \text{if $y==n$}.\\
  0, & \text{caso contrário}.
  \end{cases}$$
  
Agora o custo é definido como:
\begin{align}
J(\mathbf{w},b) = -\frac{1}{m} \left[ \sum_{i=1}^{m} \sum_{j=1}^{N}  1\left\{y^{(i)} == j\right\} \log \frac{e^{z^{(i)}_j}}{\sum_{k=1}^N e^{z^{(i)}_k} }\right] \tag{4}
\end{align}

Onde $m$ é o número de exemplos, $N$ é o número de saídas. Essa é a média de todas as perdas.


## Tensorflow
Este laboratório discutirá duas maneiras de implementar a perda de entropia cruzada softmax no Tensorflow: o método "óbvio" e o método "recomendado". O primeiro é o mais simples, enquanto o segundo é mais estável numericamente.

Vamos começar criando um conjunto de dados para treinar um modelo de classificação multiclasse.

In [ ]:
# Criar um conjunto de dados de exemplo
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)

### A organização *Óbvia*

O modelo abaixo é implementado com o softmax como ativação na camada `Dense` final.
A função de perda é especificada separadamente na diretiva `compile`. 

A função de perda é `SparseCategoricalCrossentropy`. Essa perda é descrita em (3) acima. Nesse modelo, o softmax ocorre na última camada. A função de perda recebe a saída do softmax, que é um vetor de probabilidades. 

In [ ]:
model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < ativação softmax aqui
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=10
)
        

Como o softmax é integrado à camada de saída, a saída é um vetor de probabilidades.

In [ ]:
p_nonpreferred = model.predict(X_train)
print(p_nonpreferred [:2])
print("maior valor", np.max(p_nonpreferred), "menor valor", np.min(p_nonpreferred))

### Método Recomendado <img align="Right" src="./images/C2_W2_softmax_accurate.png"  style=" width:400px; padding: 10px 20px ; ">

Relembrando a aula, resultados mais estáveis e precisos podem ser obtidos se o softmax e a perda forem combinados durante o treinamento.   Isso é possibilitado pela organização "recomendada" mostrada aqui.

Na organização recomendada, a camada final tem uma ativação linear. Por razões históricas, os outputs nesse formato são chamados de *logits*. A função de perda tem um argumento adicional: `from_logits = True`. Isso informa à função de perda que a operação softmax deve ser incluída no cálculo da perda. Isso permite uma implementação otimizada.

In [ ]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- observe
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),   #<-- observe
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
        

#### Manuseio de resultados
Observe que, no modelo recomendado, os resultados não são probabilidades, mas podem variar de grandes números negativos a grandes números positivos.A saída deve ser enviada por meio de um softmax ao realizar uma previsão que espera uma probabilidade. 
Vamos dar uma olhada nos resultados do modelo preferencial:

In [ ]:
p_preferred = preferred_model.predict(X_train)
print(f"dois exemplos de vetores de saída:\n {p_preferred[:2]}")
print("maior valor", np.max(p_preferred), "menor valor", np.min(p_preferred))

As previsões de saída não são probabilidades!
Se a saída desejada for uma probabilidade, ela deverá ser processada por um [softmax] (https://www.tensorflow.org/api_docs/python/tf/nn/softmax).

In [ ]:
sm_preferred = tf.nn.softmax(p_preferred).numpy()
print(f"dois exemplos de vetores de saída:\n {sm_preferred[:2]}")
print("maior valor", np.max(sm_preferred), "menor valor", np.min(sm_preferred))

Para selecionar a categoria mais provável, o softmax não é necessário. É possível encontrar o índice da maior saída usando [np.argmax()](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html).

In [ ]:
for i in range(5):
    print( f"{p_preferred[i]}, category: {np.argmax(p_preferred[i])}")

## SparseCategorialCrossentropy ou CategoricalCrossEntropy
O Tensorflow tem dois formatos possíveis para valores de alvo e a seleção da perda define o que é esperado.
- `SparseCategorialCrossentropy`: espera que o alvo seja um número inteiro correspondente ao índice. Por exemplo, se houver 10 valores-alvo em potencial, y estaria entre 0 e 9. 
- `CategoricalCrossEntropy`: Espera que o valor-alvo de um exemplo seja codificado _one-hot encoded_, em que o valor no índice-alvo é 1, enquanto as outras N-1 entradas são zero. Um exemplo com 10 valores-alvo potenciais, em que o alvo é 2, seria [0,0,1,0,0,0,0,0,0,0,0,0,0].


## Parabéns!
Neste laboratório, você 
- Familiarizou-se com a função softmax e seu uso na regressão softmax e nas ativações softmax em redes neurais. 
- Aprendeu a construção do modelo recomendado no Tensorflow:
    - Nenhuma ativação na camada final (o mesmo que ativação linear)
    - Função de perda SparseCategoricalCrossentropy
    - use from_logits=True
- Reconheceu que, diferentemente do ReLU e do Sigmoid, o softmax abrange várias saídas.